<a href="https://colab.research.google.com/github/okana2ki/gai4e/blob/main/emotion_sns3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データのエクスポート
投稿データや分析結果を CSV ファイルでダウンロードできるようにする。

In [1]:
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib
!apt install -y fonts-noto-cjk

import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import json, re, threading, time, queue
import tempfile
import os

FONT_PATH = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"

# --- データとキュー ---
df = pd.DataFrame(columns=["投稿","sentiment","ai_comment","status"])
post_queue = queue.Queue()
latest_ai_comment = ""

# --- 可視化関数 ---
def update_visualizations():
    global df
    # 感情ラベルの日本語マッピング
    sentiment_labels_jp = {
        "interest": "興味",
        "understanding": "理解",
        "confusion": "困惑",
        "neutral": "中立",
        "pending": "処理中"
    }
    df_display = df.copy()
    df_display['sentiment'] = df_display['sentiment'].map(sentiment_labels_jp)

    fig = px.histogram(
        df_display,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        category_orders={"sentiment":["興味","理解","困惑","中立","処理中"]},
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={"興味":"gold","理解":"lightblue","困惑":"orange","中立":"lightgrey","処理中":"lightpink"}
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white", font_path=FONT_PATH).generate(all_text)
    wc_image = wc.to_image()
    return fig, wc_image, df_display

# --- バックグラウンドLLM処理 ---
def process_queue():
    global df, latest_ai_comment
    while True:
        try:
            text, row_index = post_queue.get(timeout=1)
        except queue.Empty:
            time.sleep(1)
            continue
        system_instruction = (
            "あなたはSNS分析の専門家です。"
            "以下の文章を解析して、JSON形式で出力してください。"
            "JSONには2つのキーを含めます："
            " 1) 'sentiment' : interest, understanding, confusion, neutral のいずれか"
            " 2) 'ai_comment' : オープンキャンパスに来場した高校生を元気づけ、宮崎産業経営大学に入学したくなるようなポジティブコメント"
            "出力は必ずJSON形式のみで返してください。"
            "'ai_comment'では、経営学部に2026年度に新設されるエコノデータサイエンス学科（データサイエンスコースと実践経済コースから希望に応じて選べる）を薦めて下さい。"
        )
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
            contents=text
        )
        match = re.search(r"\{.*\}", response.text, re.DOTALL)
        if match:
            try:
                result = json.loads(match.group())
                sentiment = result.get("sentiment","neutral").strip().lower()
                ai_comment = result.get("ai_comment","")
            except:
                sentiment = "neutral"
                ai_comment = response.text
        else:
            sentiment = "neutral"
            ai_comment = response.text
        if sentiment not in ["interest","understanding","confusion","neutral"]:
            sentiment = "neutral"
        df.loc[row_index,"sentiment"] = sentiment
        df.loc[row_index,"ai_comment"] = ai_comment
        df.loc[row_index,"status"] = "分析完了"
        latest_ai_comment = ai_comment
        post_queue.task_done()
        time.sleep(4)  # 15RPM制限対応

threading.Thread(target=process_queue, daemon=True).start()

# --- 投稿処理 ---
def submit_post(text):
    global df, post_queue
    row_index = len(df)
    df.loc[row_index] = [text, "pending", "", "生成AIへのリクエスト待ち"]
    post_queue.put((text,row_index))
    fig, wc_image, df_display = update_visualizations()
    return fig, wc_image, "", "投稿完了。感情分析は順番に処理されます。", df_display, "生成AIへのリクエスト待ち", ""

# --- 更新ボタン処理 ---
def update_display(input_text_value):
    fig, wc_image, df_display = update_visualizations()
    matching_rows = df[df["投稿"] == input_text_value]
    if not matching_rows.empty:
        ai_comment = matching_rows.iloc[-1]["ai_comment"]
        sentiment_result = df_display[df_display["投稿"] == input_text_value].iloc[-1]["sentiment"]
        status = df_display[df_display["投稿"] == input_text_value].iloc[-1]["status"]
    else:
        ai_comment = ""
        sentiment_result = ""
        status = "投稿してください"
    return fig, wc_image, ai_comment, df_display, status, sentiment_result

# --- CSVダウンロード関数 ---
def download_csv():
    global df
    df_display = df.copy()
    # 感情ラベルの日本語マッピング
    sentiment_labels_jp = {
        "interest": "興味",
        "understanding": "理解",
        "confusion": "困惑",
        "neutral": "中立",
        "pending": "処理中"
    }
    df_display['sentiment'] = df_display['sentiment'].map(sentiment_labels_jp)
    with tempfile.NamedTemporaryFile(mode="w", suffix=".csv", delete=False, encoding="utf-8-sig") as tmp_file:
        df_display.to_csv(tmp_file.name, index=False)
    return tmp_file.name

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ")
    with gr.Row():
        input_text = gr.Textbox(label="感想入力")
        submit_btn = gr.Button("投稿")
        update_btn = gr.Button("表示更新")
    with gr.Row():
        ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
        sentiment_output = gr.Textbox(label="感情分析結果", interactive=False)
        status_output = gr.Textbox(label="投稿ステータス", interactive=False)
    with gr.Row():
        graph_output = gr.Plot(label="感情分析グラフ")
        wc_output = gr.Image(label="ワードクラウド")
    with gr.Row():
        dataframe_output = gr.DataFrame(label="投稿一覧")
    with gr.Row():
        download_btn = gr.Button("CSVダウンロード")
        csv_file = gr.File(label="ダウンロードファイル", file_count="single", interactive=False)


    submit_btn.click(submit_post, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, status_output, dataframe_output, status_output, sentiment_output])
    update_btn.click(update_display, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, dataframe_output, status_output, sentiment_output])
    download_btn.click(download_csv, outputs=csv_file)

demo.launch()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto-cjk is already the newest version (1:20220127+repack1-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5b9fe781628aaab892.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# 上記のスクリプトをHugging Face Spacesで公開できるよう修正
google.colab.userdataはColab環境でのみ利用可能なため、APIキーを環境変数から読み込むようにコードを変更

In [ ]:
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib
!apt install -y fonts-noto-cjk

import google.genai as genai
# from google.colab import userdata
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
# client = genai.Client(api_key=GOOGLE_API_KEY)
client = genai.Client()  # 環境変数から API Key 自動取得

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import json, re, threading, time, queue
import tempfile
import os

FONT_PATH = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"

# --- データとキュー ---
df = pd.DataFrame(columns=["投稿","sentiment","ai_comment","status"])
post_queue = queue.Queue()
latest_ai_comment = ""

# --- 可視化関数 ---
def update_visualizations():
    global df
    # 感情ラベルの日本語マッピング
    sentiment_labels_jp = {
        "interest": "興味",
        "understanding": "理解",
        "confusion": "困惑",
        "neutral": "中立",
        "pending": "処理中"
    }
    df_display = df.copy()
    df_display['sentiment'] = df_display['sentiment'].map(sentiment_labels_jp)

    fig = px.histogram(
        df_display,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        category_orders={"sentiment":["興味","理解","困惑","中立","処理中"]},
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={"興味":"gold","理解":"lightblue","困惑":"orange","中立":"lightgrey","処理中":"lightpink"}
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white", font_path=FONT_PATH).generate(all_text)
    wc_image = wc.to_image()
    return fig, wc_image, df_display

# --- バックグラウンドLLM処理 ---
def process_queue():
    global df, latest_ai_comment
    while True:
        try:
            text, row_index = post_queue.get(timeout=1)
        except queue.Empty:
            time.sleep(1)
            continue
        system_instruction = (
            "あなたはSNS分析の専門家です。"
            "以下の文章を解析して、JSON形式で出力してください。"
            "JSONには2つのキーを含めます："
            " 1) 'sentiment' : interest, understanding, confusion, neutral のいずれか"
            " 2) 'ai_comment' : オープンキャンパスに来場した高校生を元気づけ、宮崎産業経営大学に入学したくなるようなポジティブコメント"
            "出力は必ずJSON形式のみで返してください。"
            "'ai_comment'では、経営学部に2026年度に新設されるエコノデータサイエンス学科（データサイエンスコースと実践経済コースから希望に応じて選べる）を薦めて下さい。"
        )
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
            contents=text
        )
        match = re.search(r"\{.*\}", response.text, re.DOTALL)
        if match:
            try:
                result = json.loads(match.group())
                sentiment = result.get("sentiment","neutral").strip().lower()
                ai_comment = result.get("ai_comment","")
            except:
                sentiment = "neutral"
                ai_comment = response.text
        else:
            sentiment = "neutral"
            ai_comment = response.text
        if sentiment not in ["interest","understanding","confusion","neutral"]:
            sentiment = "neutral"
        df.loc[row_index,"sentiment"] = sentiment
        df.loc[row_index,"ai_comment"] = ai_comment
        df.loc[row_index,"status"] = "分析完了"
        latest_ai_comment = ai_comment
        post_queue.task_done()
        time.sleep(4)  # 15RPM制限対応

threading.Thread(target=process_queue, daemon=True).start()

# --- 投稿処理 ---
def submit_post(text):
    global df, post_queue
    row_index = len(df)
    df.loc[row_index] = [text, "pending", "", "生成AIへのリクエスト待ち"]
    post_queue.put((text,row_index))
    fig, wc_image, df_display = update_visualizations()
    return fig, wc_image, "", "投稿完了。感情分析は順番に処理されます。", df_display, "生成AIへのリクエスト待ち", ""

# --- 更新ボタン処理 ---
def update_display(input_text_value):
    fig, wc_image, df_display = update_visualizations()
    matching_rows = df[df["投稿"] == input_text_value]
    if not matching_rows.empty:
        ai_comment = matching_rows.iloc[-1]["ai_comment"]
        sentiment_result = df_display[df_display["投稿"] == input_text_value].iloc[-1]["sentiment"]
        status = df_display[df_display["投稿"] == input_text_value].iloc[-1]["status"]
    else:
        ai_comment = ""
        sentiment_result = ""
        status = "投稿してください"
    return fig, wc_image, ai_comment, df_display, status, sentiment_result

# --- CSVダウンロード関数 ---
def download_csv():
    global df
    df_display = df.copy()
    # 感情ラベルの日本語マッピング
    sentiment_labels_jp = {
        "interest": "興味",
        "understanding": "理解",
        "confusion": "困惑",
        "neutral": "中立",
        "pending": "処理中"
    }
    df_display['sentiment'] = df_display['sentiment'].map(sentiment_labels_jp)
    with tempfile.NamedTemporaryFile(mode="w", suffix=".csv", delete=False, encoding="utf-8-sig") as tmp_file:
        df_display.to_csv(tmp_file.name, index=False)
    return tmp_file.name

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ")
    with gr.Row():
        input_text = gr.Textbox(label="感想入力")
        submit_btn = gr.Button("投稿")
        update_btn = gr.Button("表示更新")
    with gr.Row():
        ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
        sentiment_output = gr.Textbox(label="感情分析結果", interactive=False)
        status_output = gr.Textbox(label="投稿ステータス", interactive=False)
    with gr.Row():
        graph_output = gr.Plot(label="感情分析グラフ")
        wc_output = gr.Image(label="ワードクラウド")
    with gr.Row():
        dataframe_output = gr.DataFrame(label="投稿一覧")
    with gr.Row():
        download_btn = gr.Button("CSVダウンロード")
        csv_file = gr.File(label="ダウンロードファイル", file_count="single", interactive=False)


    submit_btn.click(submit_post, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, status_output, dataframe_output, status_output, sentiment_output])
    update_btn.click(update_display, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, dataframe_output, status_output, sentiment_output])
    download_btn.click(download_csv, outputs=csv_file)

demo.launch()

# 処理状況を正しく知らせるように修正
合わせて、自分の投稿に対する感情分析結果も表示するように変更

In [ ]:
!pip install -q -U google-genai gradio plotly pandas wordcloud matplotlib
!apt install -y fonts-noto-cjk

import google.genai as genai
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

import pandas as pd
import gradio as gr
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import json, re, threading, time, queue

FONT_PATH = "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc"

# --- データとキュー ---
df = pd.DataFrame(columns=["投稿","sentiment","ai_comment","status"])
post_queue = queue.Queue()
latest_ai_comment = ""

# --- 可視化関数 ---
def update_visualizations():
    global df
    # 感情ラベルの日本語マッピング
    sentiment_labels_jp = {
        "interest": "興味",
        "understanding": "理解",
        "confusion": "困惑",
        "neutral": "中立",
        "pending": "処理中"
    }
    df_display = df.copy()
    df_display['sentiment'] = df_display['sentiment'].map(sentiment_labels_jp)

    fig = px.histogram(
        df_display,
        x="sentiment",
        color="sentiment",
        text_auto=True,
        category_orders={"sentiment":["興味","理解","困惑","中立","処理中"]},
        title="オープンキャンパス感想のリアルタイム感情分析",
        color_discrete_map={"興味":"gold","理解":"lightblue","困惑":"orange","中立":"lightgrey","処理中":"lightpink"}
    )
    fig.update_layout(yaxis_title="投稿数", xaxis_title="感情")
    all_text = " ".join(df["投稿"].tolist())
    wc = WordCloud(width=400, height=300, background_color="white", font_path=FONT_PATH).generate(all_text)
    wc_image = wc.to_image()
    return fig, wc_image, df_display

# --- バックグラウンドLLM処理 ---
def process_queue():
    global df, latest_ai_comment
    while True:
        try:
            text, row_index = post_queue.get(timeout=1)
        except queue.Empty:
            time.sleep(1)
            continue
        system_instruction = (
            "あなたはSNS分析の専門家です。"
            "以下の文章を解析して、JSON形式で出力してください。"
            "JSONには2つのキーを含めます："
            " 1) 'sentiment' : interest, understanding, confusion, neutral のいずれか"
            " 2) 'ai_comment' : オープンキャンパスに来場した高校生を元気づけ、この大学（宮崎産業経営大学）に入学したくなるようなポジティブコメント"
            "出力は必ずJSON形式のみで返してください。"
        )
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            config=genai.types.GenerateContentConfig(system_instruction=system_instruction),
            contents=text
        )
        match = re.search(r"\{.*\}", response.text, re.DOTALL)
        if match:
            try:
                result = json.loads(match.group())
                sentiment = result.get("sentiment","neutral").strip().lower()
                ai_comment = result.get("ai_comment","")
            except:
                sentiment = "neutral"
                ai_comment = response.text
        else:
            sentiment = "neutral"
            ai_comment = response.text
        if sentiment not in ["interest","understanding","confusion","neutral"]:
            sentiment = "neutral"
        df.loc[row_index,"sentiment"] = sentiment
        df.loc[row_index,"ai_comment"] = ai_comment
        df.loc[row_index,"status"] = "分析完了"
        latest_ai_comment = ai_comment
        post_queue.task_done()
        time.sleep(4)  # 15RPM制限対応

threading.Thread(target=process_queue, daemon=True).start()

# --- 投稿処理 ---
def submit_post(text):
    global df, post_queue
    row_index = len(df)
    df.loc[row_index] = [text, "pending", "", "生成AIへのリクエスト待ち"]
    post_queue.put((text,row_index))
    fig, wc_image, df_display = update_visualizations()
    return fig, wc_image, "", "投稿完了。感情分析は順番に処理されます。", df_display, "生成AIへのリクエスト待ち", ""

# --- 更新ボタン処理 ---
def update_display(input_text_value):
    fig, wc_image, df_display = update_visualizations()
    matching_rows = df[df["投稿"] == input_text_value]
    if not matching_rows.empty:
        ai_comment = matching_rows.iloc[-1]["ai_comment"]
        sentiment_result = df_display[df_display["投稿"] == input_text_value].iloc[-1]["sentiment"]
        status = df_display[df_display["投稿"] == input_text_value].iloc[-1]["status"]
    else:
        ai_comment = ""
        sentiment_result = ""
        status = "投稿してください"
    return fig, wc_image, ai_comment, df_display, status, sentiment_result

# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# オープンキャンパス感想SNSデモ")
    input_text = gr.Textbox(label="感想入力")
    submit_btn = gr.Button("投稿")
    update_btn = gr.Button("集計更新")
    ai_comment_output = gr.Textbox(label="AIコメント", interactive=False)
    sentiment_output = gr.Textbox(label="感情分析結果", interactive=False)
    status_output = gr.Textbox(label="投稿ステータス", interactive=False)
    graph_output = gr.Plot(label="感情分析グラフ")
    wc_output = gr.Image(label="ワードクラウド")
    dataframe_output = gr.DataFrame(label="投稿一覧")

    submit_btn.click(submit_post, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, status_output, dataframe_output, status_output, sentiment_output])
    update_btn.click(update_display, inputs=input_text, outputs=[graph_output, wc_output, ai_comment_output, dataframe_output, status_output, sentiment_output])

demo.launch()